In [1]:
import numpy as np

In [2]:
from load_contracts import read_contract, preprocess_text
from load_info import get_security_names

In [48]:
filename = "contracts/135_ActelisNetworks_COI_01072005.pdf"

In [67]:
END_OF_IV_INTRO_TEXT = ['rel','right','prefer','privileg','and','restrict','grant','to','or','impos','upon','the','common','stock','and','the','prefer','stock','and','the','holder']

In [53]:
text = read_contract(filename)
text = text.split(" ")
text = list(filter(lambda a: a != '', text))

In [81]:
def get_closest(values, target, less, any=False):
    if (any):
        val = min(values, key=lambda x: abs(target - x))
    elif (less):
        val = min(values, key=lambda x: target - x)
    else:
        val = min(values, key=lambda x: x - target)
    return val

In [35]:
def find_target(text, target):
    target_loc = 0
    indicies = []
    start_index = 0
    for i in range(len(text)):
        word = text[i]
#         print("target_loc", target_loc)
        if word == target[target_loc]:
            if target_loc == 0: #marks start of sequence
                start_index = i
            target_loc += 1
            if target_loc == len(target):
                indicies += [start_index]
                target_loc = 0
        elif word == target[0]:
            target_loc = 1
        else:
            if target_loc > 0:
                #reset just incase ababcd 
                i -= (target_loc - 1) 
            target_loc = 0
    return indicies

In [36]:
x = find_target(text, ["common"])

In [133]:
def get_security_types(text):
    unsplit_names = get_security_names()
    names = [name.split(" ") for name in unsplit_names]
    indicies = []
    types = []
    for name in names:
        if (len(name) == 1):
            name += ["stock"]
        cur_indicies = find_target(text, name)
        if len(cur_indicies) > 0:
            types += [" ".join(name)]
            indicies += [cur_indicies]
    #remove extra "prefered stock"
    has_extra_prefered = True
    if "prefer stock" in types:
        for type in types:
            if type[-6:] == 'prefer':
                has_extra_prefered = True
                break
        if (has_extra_prefered):
            indicies.pop(types.index('prefer stock'))
            types.remove('prefer stock')
    return indicies, types

In [37]:
unsplit_names = get_security_names()

In [38]:
names = [name.split(" ") for name in unsplit_names]

In [39]:
indicies = []
for name in names:
    indicies += [find_target(text, name)]


In [59]:
index_of_article_IV = find_target(text, ["articl", "fourth"])

In [60]:
text[index_of_article_IV[0]: index_of_article_IV[0] + 2]

['articl', 'fourth']

### Search Through IV Intro

In [82]:
end_of_IV_intro = find_target(text, END_OF_IV_INTRO_TEXT)

In [83]:
assert(len(end_of_IV_intro) == 1)

In [86]:
end_of_IV_intro = end_of_IV_intro[0]

In [92]:
articl_forth_occur = find_target(text, ["articl", "fourth"])

In [93]:
begining_of_IV = get_closest(values = articl_forth_occur, target=end_of_IV_intro, less=True)

In [94]:
print("IV Intro starts", begining_of_IV, "IV Intro ends", end_of_IV_intro)

IV Intro starts 203 IV Intro ends 363


In [98]:
" ".join(text[begining_of_IV:end_of_IV_intro])

'articl fourth compani is author to issu two class of share design respect common stock and prefer stock the compani is author to issu 43000000 share of common stock with a par valu of $0.01 per share the compani is author to issu 26007500 share of prefer stock 800000 of which are design seri a prefer stock with a par valu of $0.01 per share seri a prefer 700000 of which are design seri b prefer stock with a par valu of $0.01 per share seri b prefer 1507500 of which are design seri c prefer stock with a par valu of $0.01 per share seri c prefer and 23000000 of which are design seri d prefer stock with a par valu of $0.01 per share seri d prefer for the purpos of thi amend and restat certif of incorpor the term prefer stock shall mean seri a prefer seri b prefer seri c prefer and seri d prefer collectively.th'

In [131]:
IV_into_text = text[begining_of_IV:end_of_IV_intro]

In [134]:
indicies, types = get_security_types(IV_into_text)

In [135]:
types

['common stock',
 'seri a prefer',
 'seri b prefer',
 'seri c prefer',
 'seri d prefer']